In [1]:
# Set start time

from datetime import datetime
overall_start_time = datetime.now()

In [2]:
# Import the data
from tensorflow.keras import layers, models, Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import torch

In [3]:
# PyTorch Environment 

print(torch.backends.cudnn.enabled)

print(torch.cuda.is_available())

print(torch.cuda.device_count())

!python -m torch.utils.collect_env

from timeit import default_timer as timer

# check for cuda availability
print("Cuda: ", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)


#GPU 
b = torch.ones(4000,4000).cuda() # Create matrix on GPU memory
start_time = timer() 
for _ in range(1000): 
    b += b 
elapsed_time = timer() - start_time 

print('GPU time = ',elapsed_time)


#CPU
a = torch.ones(4000,4000) # Create matrix on CPU memory
start_time = timer()
for _ in range(1000):
    a += a
elapsed_time = timer() - start_time

print('CPU time = ',elapsed_time)

True
True
1
PyTorch version: 2.0.1+cu117
Is debug build: False
CUDA used to build PyTorch: 11.7
ROCM used to build PyTorch: N/A

OS: Microsoft Windows 11 Home
GCC version: Could not collect
Clang version: Could not collect
CMake version: Could not collect
Libc version: N/A

Python version: 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)] (64-bit runtime)
Python platform: Windows-10-10.0.22631-SP0
Is CUDA available: True
CUDA runtime version: 12.5.40

CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
Nvidia driver version: 555.85
cuDNN version: Could not collect
HIP runtime version: N/A
MIOpen runtime version: N/A
Is XNNPACK available: True

CPU:
Architecture=9


CurrentClockSpeed=2300


DeviceID=CPU0


Family=198


L2CacheSize=9728


L2CacheSpeed=


Manufacturer=GenuineIntel


MaxClockSpeed=2300


Name=12th Gen Intel(R) Core(TM) i7-12650H


ProcessorType=3


Revision=

Versions of rel

In [4]:
# Load the CSV file
CO_crashes_df_updated = pd.read_csv('./Resources/result_files/step1_build_base_data.csv')

In [5]:
# Drop Street (not used)
CO_crashes_df_updated = CO_crashes_df_updated.drop(columns=['Street']  )

In [6]:
# Display the first five rows of the DataFrame
CO_crashes_df_updated.head()

,Unnamed: 0,County,Severity,Start_Time,Weather_Condition,Start_Time_int,ID
0,0,1,3,2022-09-08 13:03:18,Partly Cloudy,20220908,57335
1,1,59,1,2022-09-08 08:44:08,Fair,20220908,57362
2,2,1,2,2022-09-08 06:57:38,Fair,20220908,57373
3,3,5,1,2022-09-08 08:45:51,Fair,20220908,57374
4,4,31,3,2022-09-08 08:42:23,Fair,20220908,57375


In [7]:
# Preprocess "Weather_Condition" column (one-hot encoding)
weather_encoder = OneHotEncoder(sparse_output=False)
weather_encoded = weather_encoder.fit_transform(CO_crashes_df_updated[['Weather_Condition']])
weather_columns = weather_encoder.get_feature_names_out(['Weather_Condition'])
df_weather_encoded = pd.DataFrame(weather_encoded, columns=weather_columns)
df_weather_encoded.head(5)

,Weather_Condition_Cloudy,Weather_Condition_Cloudy Windy,Weather_Condition_Cloudy _ Windy,Weather_Condition_Drizzle,Weather_Condition_Fair,Weather_Condition_Fair _ Windy,Weather_Condition_Fog,Weather_Condition_Haze,Weather_Condition_Heavy Snow,Weather_Condition_Light Drizzle,...,Weather_Condition_Rain,Weather_Condition_Showers in the Vicinity,Weather_Condition_Smoke,Weather_Condition_Snow,Weather_Condition_T_Storm,Weather_Condition_Thunder,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
CO_crashes_df_updated['Weather_Condition'].value_counts()

Fair                       32024
Mostly Cloudy              16447
Partly Cloudy              13865
Cloudy                     12382
Light Snow                  6424
Snow                        1145
Fair _ Windy                1064
Light Rain                  1024
Fog                          748
Mostly Cloudy _ Windy        574
Partly Cloudy _ Windy        473
Haze                         430
Light Drizzle                404
Heavy Snow                   382
Rain                         276
Light Snow _ Windy           242
Cloudy _ Windy               234
Thunder in the Vicinity      217
Showers in the Vicinity      205
Smoke                        202
Light Rain with Thunder      166
Thunder                      156
T_Storm                      156
Wintry Mix                   154
Light Snow Windy              48
Cloudy Windy                  16
Windy                         12
Light Snow  Windy              2
Light Snow Showers             2
Drizzle                        1
Name: Weat

In [9]:
# define X
X = CO_crashes_df_updated.drop(columns=['County', 'Weather_Condition', 'Start_Time', 'ID'])

df_weather_encoded = pd.concat([CO_crashes_df_updated, df_weather_encoded], axis=1)

# Define Y

df_weather_encoded['County'] = CO_crashes_df_updated['County']

y_county = df_weather_encoded['County']

y_weather = df_weather_encoded[weather_columns]

# Split data into training and testing sets

X_train, X_test, y_county_train, y_county_test, y_weather_train, y_weather_test = train_test_split(X, y_county, y_weather)

X.head()

,Unnamed: 0,Severity,Start_Time_int
0,0,3,20220908
1,1,1,20220908
2,2,2,20220908
3,3,1,20220908
4,4,3,20220908


In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Create the shared layers of the model

# Input layer
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [12]:
# Branch for weather prediction
weather_output = layers.Dense(31, activation='softmax', name='weather_output')(shared_layer2)

# Branch for county prediction
county_output = layers.Dense(1, activation='sigmoid', name='county_output')(shared_layer2)

In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[weather_output, county_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'county_output': 'categorical_crossentropy', 'weather_output': 'binary_crossentropy'},
              metrics={'county_output': 'accuracy', 'weather_output': 'accuracy'})

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 3)]                  0         []                            
 )                                                                                                
                                                                                                  
 dense (Dense)               (None, 64)                   256       ['input_features[0][0]']      
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 weather_output (Dense)      (None, 31)                   1023      ['dense_1[0][0]']         

In [14]:
# Fit the model
start_time = timer()
model.fit(
    X_train_scaled,
    {'county_output': y_county, 'weather_output': y_weather},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)
elapsed_time = timer() - start_time

print('Fit Model Time = ',elapsed_time)

Epoch 1/10


c:\Users\Thomas\anaconda3\envs\dev\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1683/1705 [============================>.] - ETA: 0s - loss: 0.2053 - weather_output_loss: 0.2053 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.2271 - county_output_accuracy: 0.1395

c:\Users\Thomas\anaconda3\envs\dev\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1705/1705 [==============================] - 3s 1ms/step - loss: 0.2067 - weather_output_loss: 0.2067 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.2270 - county_output_accuracy: 0.1396 - val_loss: 0.5775 - val_weather_output_loss: 0.5775 - val_county_output_loss: 0.0000e+00 - val_weather_output_accuracy: 0.0926 - val_county_output_accuracy: 0.1345
Epoch 2/10
1705/1705 [==============================] - 2s 1ms/step - loss: 0.6732 - weather_output_loss: 0.6732 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.2068 - county_output_accuracy: 0.1402 - val_loss: 2.0698 - val_weather_output_loss: 2.0698 - val_county_output_loss: 0.0000e+00 - val_weather_output_accuracy: 0.2271 - val_county_output_accuracy: 0.1345
Epoch 3/10
1705/1705 [==============================] - 2s 1ms/step - loss: 1.5428 - weather_output_loss: 1.5428 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.2039 - county_output_accuracy: 0.1402 - val_loss: 3.4248 - val_weather_output_lo

In [15]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'county_output': y_county_test, 'weather_output': y_weather_test})
test_results

 69/711 [=>............................] - ETA: 0s - loss: 100174248.0000 - weather_output_loss: 100174248.0000 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.3501 - county_output_accuracy: 0.1413  

c:\Users\Thomas\anaconda3\envs\dev\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


711/711 [==============================] - 1s 726us/step - loss: 95499480.0000 - weather_output_loss: 95499480.0000 - county_output_loss: 0.0000e+00 - weather_output_accuracy: 0.3501 - county_output_accuracy: 0.1404


[95499480.0, 95499480.0, 0.0, 0.35014522075653076, 0.14039257168769836]

In [16]:
# Print the accuracy
print(f"Weather Accuracy: {test_results[3]}")
print(f"County Accuracy: {test_results[4]}")

Weather Accuracy: 0.35014522075653076
County Accuracy: 0.14039257168769836


In [17]:
# Calculate Duration
overall_end_time = datetime.now()
print('Duration: {}'.format(overall_end_time - overall_start_time))

Duration: 0:00:39.391312
